In [1]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from data_provider.data_loader import HuaweiDataset
from models.DLinear import DLinear

from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.optimizer import Adam
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
dataset = HuaweiDataset(path="/home/user/suzhao/BehaviorDL/dataset/Huawei", label_flag='emotion')

timesteps = 1441
num_features = 4

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [3]:
dls = DataLoaders(train_loader, val_loader)
input_dim = timesteps * num_features
model = DLinear(input_dim=input_dim, hidden_dim=128, output_dim=2).to(device)  # Adjust input_dim accordingly

learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), opt_func= Adam, metrics=accuracy)
learn.fit(10)

TypeError: 'module' object is not callable

In [10]:
# Initialize the model, criterion and optimizer
input_dim = timesteps * num_features
model = DLinear(input_dim=input_dim, hidden_dim=128, output_dim=2).to(device)  # Adjust input_dim accordingly
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for data, labels in train_loader:
        data, labels = data.view(-1, timesteps*num_features).float().to(device), labels.long().to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels.squeeze())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, labels in val_loader:
        data, labels = data.view(-1, timesteps*num_features).float().to(device), labels.long().to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.squeeze()).sum().item()

print(f"Validation Accuracy: {100 * correct / total}%")

Epoch 1, Loss: 0.43816827644001355
Epoch 2, Loss: 0.4000668850812045
Epoch 3, Loss: 0.40384880792010913
Epoch 4, Loss: 0.3980443369258534
Epoch 5, Loss: 0.4020234969529239
Epoch 6, Loss: 0.40327525951645593
Epoch 7, Loss: 0.4018088768828999
Epoch 8, Loss: 0.40171464193951
Epoch 9, Loss: 0.4002454335039312
Epoch 10, Loss: 0.40175739472562616
Validation Accuracy: 92.26190476190476%
